In [1]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine

In [2]:
import nltk

nltk.download('punkt')
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\taleb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\taleb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
import os

# Obtener el directorio de trabajo actual
directorio_actual = os.getcwd()
print(directorio_actual)

c:\Users\taleb\Desktop\ULB\STIC-B545 - Traitement automatique de corpus\TP\tac\tps\tp3


In [9]:
data_path = "../../data/txt/"

1. Choisissez une décennie (1890–1899, 1900–1909, 1910–1919, etc.)

In [10]:
DECADE = '1960'

2. Effectuez un clustering des documents de cette décennie grâce au notebook s2_clustering.ipynb, en adaptant éventuellement le nombre de
clusters désirés


In [11]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]